# Deploying infersent as web service to Azure Kubernetes Service (AKS)

This notebook shows the steps for deploying a service: registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. We then test and delete the service, image and model.


In [ ]:
import os
import requests
import sys
sys.path.append("../../../")

import azureml.core
from azureml.core import Experiment, Workspace
from azureml.core.image import ContainerImage
from azureml.core.model import Model
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.exceptions import ProjectSystemException, UserErrorException


In [ ]:
# Check core SDK version number
print(f"Azure ML SDK Version: {azureml.core.VERSION}")

## Get workspace

Load existing workspace from the config file info

In [ ]:
# Let's define these variables here - These pieces of information can be found on the portal
subscription_id = os.getenv("SUBSCRIPTION_ID", default="<our_subscription_id>")
resource_group =  os.getenv("RESOURCE_GROUP", default="<our_resource_group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<our_workspace_name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="<our_workspace_region>")

try:
    # Let's load the workspace from the configuration file
    ws = Workspace.from_config()
    print("Workspace was loaded successfully from the configuration file")
except (UserErrorException, ProjectSystemException):
    # or directly from Azure, if it already exists (exist_ok=True).
    # If it does not exist, let's create a workspace from scratch
    ws = Workspace.create(name=workspace_name,
                          subscription_id=subscription_id,
                          resource_group=resource_group,
                          location=workspace_region,
                          create_resource_group=True,
                          exist_ok=True
                         )
    ws.write_config()
    print("Workspace was loaded successfully from Azure")

In [ ]:
# Print the workspace attributes
print(f'Workspace name: {ws.name}\n \
      Azure region: {ws.location}\n \
      Subscription id: {ws.subscription_id}\n \
      Resource group: {ws.resource_group}')

## Register the model 
Register an existing trained model, add descirption and tags.

In [ ]:
model = Model.register(
    model_path = "../../../data/infersent",
    model_name = "infersent",
    tags = {"Model": "InferSent"},
    description = "InferSent pretrained model",
    workspace = ws
)

## Create an image
Create an image using the registered model the script that will load and run the model.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Image with infersent model",
                                                  tags = {'area': "nlp", 'type': "sentencesimilarity"})

image = ContainerImage.create(name = "infersentimage",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)


If the above step fails then use below command to see logs 

In [ ]:

print(image.image_build_log_uri) 

## Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
# create aks cluser

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'nlp-aks' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

## Deploy web service to AKS

In [ ]:
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [ ]:

# deploy image as web service
aks_service_name ='aks-service-9'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print("----------------------------")
print(aks_service.state)

## Test the web service using run method 
We test the web sevice by passing data. Run() method retrieves API keys behind the scenes to make sure that call is authenticated. The run method expects only the input in dictionary format. 

In [ ]:
sentences = []
with open('samples.txt') as f:
    for line in f:
        sentences.append(line.strip())
print(len(sentences))

In [ ]:

sentences = ['This is sentence 1', 'This is sentence 2']

data = {"sentences": sentences}
print(data['sentences'])

In [ ]:

embeddings = aks_service.run(input_data = data)

print('nb sentences encoded : {0}'.format(len(embeddings)))
print(embeddings)

## Clean up
Delete the service, image and model.

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()